# **Projeto WaRP - Waste Recycling Plant**

---
## **⚠️ Observações Importantes:**
**1.** Todas as análises devem ter tratamento de erro, garantindo que o notebook continue rodando mesmo que ocorram problemas como dados faltantes, diretórios incorretos ou arquivos corrompidos.

**2.** Registrar qualquer problema encontrado, como:
- Alto número de imagens corrompidas.
- Desbalanceamento severo nas classes.
- Alto número de duplicatas.

**3.** Sempre documentar o que foi observado em cada gráfico ou análise.

---

## **Etapa 1 - Análise do Dataset**

### 🧠 Instalação dos pacotes necessários

In [1]:
!pip install opendatasets
!pip install pandas
!pip install imagehash
!pip install matplotlib
!pip install opencv-python
!pip install pillow
!pip install kaggle

### 1. Download do Dataset 🚀

In [2]:
# os arquivos serão salvos na pasta "data" localmente. 
# A pasta "data" local foi sincronizada bidirecionalmente com Google Drive

# Configuração das credenciais
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Verificar se o arquivo kaggle.json existe no caminho padrão
kaggle_path = os.path.expanduser('~/.kaggle/kaggle.json') # O arquivo kaggle.json deve estar salvo em C:/Users/SeuUsuario/.kaggle/kaggle.json
if not os.path.exists(kaggle_path):
    raise FileNotFoundError(f"Arquivo kaggle.json não encontrado em {kaggle_path}. Verifique se está salvo corretamente.")

# Verificar ou criar a pasta de destino
download_path = '../data' # pasta local
if not os.path.exists(download_path):
    os.makedirs(download_path)
    print(f"Pasta {download_path} criada com sucesso!")

# Fazer download do dataset
try:
    print("Autenticando com a API do Kaggle...")
    api = KaggleApi()
    api.authenticate()

    print("Iniciando o download do dataset...")
    api.dataset_download_files(
        'parohod/warp-waste-recycling-plant-dataset',
        path=download_path,
        unzip=True
    )

    print("Download concluído com sucesso!")

except Exception as e:
    print(f"Ocorreu um erro durante o download: {e}")

Autenticando com a API do Kaggle...
Iniciando o download do dataset...
Dataset URL: https://www.kaggle.com/datasets/parohod/warp-waste-recycling-plant-dataset
Download concluído com sucesso!


### 2. Criação do DataFrame de Metadados 📂

In [3]:
import os
import pandas as pd
import cv2
from PIL import Image
import imagehash
import matplotlib.pyplot as plt
from IPython.display import display

root_dir = '../data/Warp-C/train_crops' # se estiver trabalhando com pasta local no vscode
# root_dir = '/content/drive/data/Warp-C/train_crops' # se estiver trabalhando com pasta do google drive no google colab

if not os.path.exists(root_dir):
    raise FileNotFoundError(f"Diretório não encontrado: {root_dir}. Verifique o caminho.")

count_corrupted = 0
corrupted = []
dataframe_list = []

print("Iniciando a análise das imagens...")

try:
    for main_folder in os.listdir(root_dir):
        main_path = os.path.join(root_dir, main_folder)
        if os.path.isdir(main_path):
            for sub_folder in os.listdir(main_path):
                sub_path = os.path.join(main_path, sub_folder)
                if os.path.isdir(sub_path):
                    for image in os.listdir(sub_path):
                        img_path = os.path.join(sub_path, image)

                        try:
                            image_format = image.split('.')[-1].lower()
                        except IndexError:
                            image_format = None

                        try:
                            img = cv2.imread(img_path)
                            if img is not None:
                                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                                img_pil = Image.fromarray(img_rgb)
                                img_hash = imagehash.phash(img_pil)
                                width, height, channels = img.shape
                                img_corrupted = False
                            else:
                                count_corrupted += 1
                                corrupted.append(img_path)
                                img_corrupted = True
                                img_hash, width, height, channels = None, None, None, None
                        except Exception:
                            count_corrupted += 1
                            corrupted.append(img_path)
                            img_corrupted = True
                            img_hash, width, height, channels = None, None, None, None
                            print(f"Erro ao processar {img_path}: {e}")

                        dataframe_list.append([
                            img_path, img_corrupted, img_hash, image_format,
                            width, height, channels, sub_folder
                        ])

except Exception as e:
    print(f"Ocorreu um erro na varredura das imagens: {e}")
    raise                        
                        
# Criando o DataFrame
df = pd.DataFrame(dataframe_list, columns=[
    'image_path', 'corrupted', 'image_hash', 'image_format',
    'width', 'height', 'channels', 'label'
])

# df.head()
display(df.head())

#  Prints
print(f"\nTotal de imagens processadas: {len(df)}")
print(f"Total de imagens corrompidas: {count_corrupted}")
print(f"Imagens corrompidas:\n{corrupted}")

Iniciando a análise das imagens...


,image_path,corrupted,image_hash,image_format,width,height,channels,label
0,../data/Warp-C/train_crops\bottle\bottle-blue\...,False,aecead9899399491,jpg,409,190,3,bottle-blue
1,../data/Warp-C/train_crops\bottle\bottle-blue\...,False,94a724b1adbd4bc8,jpg,285,368,3,bottle-blue
2,../data/Warp-C/train_crops\bottle\bottle-blue\...,False,96cc6b62978ec272,jpg,398,424,3,bottle-blue
3,../data/Warp-C/train_crops\bottle\bottle-blue\...,False,cc8c2e3bd93469d2,jpg,375,239,3,bottle-blue
4,../data/Warp-C/train_crops\bottle\bottle-blue\...,False,df4028ff2410eecb,jpg,212,324,3,bottle-blue



Total de imagens processadas: 8823
Total de imagens corrompidas: 0
Imagens corrompidas:
[]


### 3. Verificar a Integridade dos Arquivos 🔍

#### Informações estatísticas dos valores de altura, largura e canais das imagens:

In [ ]:
# Exibir estatísticas:
# Gerar estatísticas descritivas (mean, std, min, max) sobre: Largura (width), Altura (height), Canais (channels) etc
# Verificar se existem padrões, outliers ou inconsistências nas dimensões.

#### Quantidade de imagens por formato:

In [ ]:
# Plotar gráfico:
# Plotar um gráfico de barras com a quantidade de imagens por formato de arquivo (ex.: JPG, PNG, etc.).
# Verificar se há formatos inconsistentes ou pouco utilizados.

### 4. Verificar a Consistência dos Metadados 🔧

#### Distribuição das dimensões (width x height):

In [ ]:
# Plotar o gráfico de dispersão:
# Plotar um gráfico de dispersão (scatterplot) das dimensões (width x height).
# Avaliar se as imagens possuem tamanhos muito variados ou padrões.

#### Informações nulas:

In [ ]:
# Verificar informações nulas:
# Gerar uma tabela com a quantidade de valores nulos por coluna no dataframe.
# Verificar se há dados faltantes que precisam ser tratados.

### 5. Analisar a Qualidade das Imagens 🧹

#### Total de imagens corrompidas:

In [ ]:
# Realizar a contagem de imagens corrompidas:
# Contar o total de imagens que não puderam ser abertas ou estão corrompidas.
# Validar se o número é significativo e se é necessário substituí-las ou removê-las.

#### Quantidade de imagens corrompidas e não corrompidas:

In [ ]:
# Exibir a quantidade de imagens corrompidas e não corrompidas:
# Gerar uma contagem e exibir de forma tabular (ou gráfico, se desejar) a proporção de imagens válidas e corrompidas.

### 6. Verificar a Distribuição das Classes 📊 

In [ ]:
# Plotar gráfico da distribuição das classes:
# Gerar um gráfico de barras com a quantidade de imagens por classe.
# Avaliar se há desequilíbrio nas classes, o que pode impactar o treinamento de modelos.

### 7. Verificar a Presença de Duplicatas 🧠 

#### Quantidade de duplicatas:

In [ ]:
# Calcular quantidade de duplicatas:
# Calcular e exibir a quantidade de imagens duplicadas com base no hash perceptual das imagens.


#### Visualizar duplicatas:

In [ ]:
# Verificar e exibir duplicatas, se houver duplicatas:
# Exibir uma tabela com as informações das imagens duplicadas (paths, labels, hashes).

#### Exibir imagens duplicadas:

In [ ]:
# Verificar e exibir duplicatas lado a lado, se houver duplicatas:
# Mostrar visualmente as imagens duplicadas em uma mesma linha para comparação.
# Avaliar se essas duplicatas devem ser removidas do dataset.

#### Gráficos de Duplicatas por Classe:

In [ ]:
# Plotar grafico de duplicatas por classe, se houver duplicatas:
# Plotar um gráfico de barras indicando quantas duplicatas existem por classe.
# Verificar se há classes mais impactadas por duplicações.